In [1]:
import requests
import json
import os
import datetime
import time
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()
import urllib
import time

In [4]:
def get_fb_token(app_id, app_secret):           
    payload = {'grant_type': 'client_credentials', 
               'client_id': app_id, 
               'client_secret': app_secret}
    file = requests.post('https://graph.facebook.com/oauth/access_token?', 
                         params = payload)
    result = file.text.split("=")[1]
    return result

def query_url(pageid, q, token):
    return "https://graph.facebook.com/v2.6/%s?%s&access_token=%s"%(pageid, q, token)

pageid = '136845026417486' #Ke
# pageid = '46251501064' #Tsai
# pageid = '188311137478' #ChinaTimes 
# pageid = '241284961029' #udn 
# pageid = '394896373929368' #ltn 
# pageid = '232633627068' #AppleDaily 

app_id = '1577374979150890'
app_secret = '6408b3bcaabd6a87d65200976f76a387'

token = 'EAACEdEose0cBABX10l1ZC7RIhSY1NGIrbHKL3oCQAdkZCJGbBl5YDlBZB4es4e9GOhJqXZCzXZBPHSXqw0a3XSkQZBaMtpOOItGF2H5xEzfEIa7FS0BhAiJImIcH6a2nLgtk8vZBf7sPzb6ZBs7LLcfpypp4bsM3ZBUm8bbWd6mdhT0Te1TmAk4v9'
token = get_fb_token(app_id, app_secret)

In [ ]:
directory = "data/%s/"%pageid

if not os.path.exists(directory):
    os.makedirs(directory)

# 抓取貼文的 分享數量、內文、連結、時間
url = query_url(pageid, 'fields=posts{shares,message,permalink_url,created_time}', token)
results = requests.get(url)
data = json.loads(results.text)
posts = data['posts']['data']

# for q,w in enumerate(data['posts']['data']):
#     print q # 0~24
#     likes_id = []
#     url = query_url(p['id']+'/likes', %s, 'fields=id{link,comments.limit(%d)}'%(amount, maxcc), token)

# print data['posts']['paging']['next']

url = data['posts']['paging']['next']

# pids = [p['id'] for p in data['posts']['data']]
while True:
    results = requests.get(url)
    data = json.loads(results.text)
    if len(data['data']) == 0:
        break    
    posts += data['data']
#     print data['paging']['next']
    url = data['paging']['next']
    print 'Total post:%d'%len(posts)

print len(posts)
fname = directory + 'posts.json'
json.dump(posts, open(fname, 'w'))

Total post:50
Total post:75
Total post:100
Total post:125
Total post:150
Total post:175
Total post:200
Total post:225
Total post:250
Total post:275
Total post:300
Total post:325
Total post:350
Total post:375
Total post:400
Total post:425
Total post:450
Total post:475
Total post:500
Total post:525
Total post:550
Total post:575
Total post:600
Total post:625
Total post:650
Total post:675
Total post:700
Total post:725
Total post:750
Total post:775
Total post:800
Total post:825
Total post:850
Total post:875
Total post:900
Total post:925
Total post:950
Total post:975
Total post:1000
Total post:1025
Total post:1050
Total post:1075
Total post:1100
Total post:1125
Total post:1150
Total post:1175
Total post:1200
Total post:1225
Total post:1250
Total post:1275
Total post:1300
Total post:1325
Total post:1350
Total post:1375
Total post:1400
Total post:1425
Total post:1450
Total post:1475
Total post:1500
Total post:1525
Total post:1550
Total post:1575
Total post:1600
Total post:1625
Total post:1650


抓取回應部份

In [5]:
directory = "data/%s/"%pageid
post_path = "data/%s/posts.json"%pageid
import operator
posts = []
with open(post_path) as ff:
    for line in ff:
        posts.append(json.loads(line))

s_posts=[]

for i in range(len(posts[0])):
    s_posts.append(posts[0][i])
print len(s_posts)

1090


## 從文章的編號去挖

In [7]:
start = time.time()
DEBUG = False
amount = 50 if DEBUG else 500
# print "Length of posts: %d"%len(posts)

for i, p in enumerate(s_posts): #取出每一則貼文的ip再丟進去查
#     print i, p['id']
    if i in range(313,1091):
        print i, p['id']
        url = query_url(p['id'], 'fields=reactions.summary(true),comments.limit(%d){comment_count,like_count,created_time,from,message}'%amount, token)
        print url
        results = requests.get(url)
        data = json.loads(results.text)
#         print data.keys()
        if len(data.keys()) == 1: 
            print "ERROR:CANNOT GET THE POST:", data.keys(), p['id']
            continue
        #抓取comments
        if 'comments' in data:
            c_data = data['comments']
            comments = c_data['data']
            
            if 'next' in c_data['paging']:
                    url = c_data['paging']['next']
                    results = requests.get(url)
#                     print url
                    n_data = json.loads(results.text)
#                     print n_data
                    comments +=n_data['data']
#                     try:
#                         comments += c_data['data']
#                     except:
#                         time.sleep(3)
#                         print "ERROR: time sleep 3 seconds"
#                         print data.keys()
#                         comments  += c_data['data']
                    if 'next' not in c_data['paging']:
                        break

# #             print 'PostID[%d] %s (comments:%d)  like_by_id: %d time:%d'%(i, p['id'], len(comments), len(likes_list),time.time()-start)
# #             print p['permalink_url']

#             fname1 = directory + 'comment%d-%s.json'%(i, p['id'])
#             with open(fname1, 'w') as fout:
#                 json.dump(comments, fout)

            #以下為抓取reaction 部分
            maxl = max(c['like_count'] for c in comments)
        #     print maxl
            if DEBUG: print('maxcc:%d'%maxcc)
        #     url = query_url(p['id'], 'fields=likes.limit(%d){likes.limit(%d)}'%(amount, maxl), token)
            url = query_url(p['id'], 'fields=reactions.limit(%d){reactions.limit(%d),id,type}'%(amount, maxl), token)
            results = requests.get(url)
            data = json.loads(results.text)
            if len(data) == 1:
                reac_list.append(data)
            else:
                data = data['reactions']
                reac_list = []
                try:
                    if not 'next' in data['paging']:
                        reac_list.append(data['data'])
                    if 'next' in data['paging']:
                        while True: #如果有出現“next"，那就再去抓取下一頁  
                #             print data.keys()
                            url = data['paging']['next']
                            results = requests.get(url)
                            data = json.loads(results.text)
                            try:    
                                reac_list.append(data)
                            except:
                                time.sleep(3)
                                print "ERROR: time sleep 3 seconds"
#                                 print data.keys()
                                reac_list.append(data) 
                            if 'next' not in data['paging']:
                                break
                except:
                    print i, p['id'], data
            print 'PostID[%d] %s (reaction_by_id:%d) time:%d'%(i, p['id'], len(reac_list[0]),time.time()-start)
            fname = directory + 'reactions%d-%s.json'%(i, p['id'])
            with open(fname, 'w') as fout:
                json.dump(reac_list, fout)


        if not 'comments' in data:
            if 'reactions' in data:

                r_data = data['reactions']
                reply = r_data['data']
                maxl = r_data['summary']['total_count']
        #         if DEBUG: print('maxcc:%d'%maxcc)
            #     url = query_url(p['id'], 'fields=likes.limit(%d){likes.limit(%d)}'%(amount, maxl), token)
                url = query_url(p['id'], 'fields=reactions.limit(%d){reactions.limit(%d),id,type}'%(amount, maxl), token)
                results = requests.get(url)
                data = json.loads(results.text)
                print data.keys()
    #             data = data['reactions']
                reac_list = []
                try:
                    if not 'next' in data['paging']:
                        url = query_url(p['id'], 'fields=reactions.limit(%d){reactions.limit(%d),id,type}'%(amount, maxl), token)
            #             url = query_url(p['id'], 'fields=reactions{id,type}'%amount, token)
                        reac_list.append(data['data'])
                    if 'next' in data['paging']:
                        while True: #如果有出現“next"，那就再去抓取下一頁  
                #             print data.keys()
                            url = data['paging']['next']
                            results = requests.get(url)
                            data = json.loads(results.text)
                            try:    
                                reac_list.append(data)
                            except:
                                time.sleep(3)
                                print "ERROR: time sleep 3 seconds"
                                print data.keys()
                                reac_list.append(data) 
                            if 'next' not in data['paging']:
                                break
                except:
                    print i, p['id'],data.keys()
                    reac_list.append([''])
                print 'PostID[%d] %s (reaction_by_id:%d) time:%d'%(i, p['id'], len(reac_list[0]),time.time()-start)
                fname = directory + 'reactions%d-%s.json'%(i, p['id'])
                with open(fname, 'w') as fout:
                    json.dump(reac_list, fout)


313 136845026417486_667080010060649
https://graph.facebook.com/v2.6/136845026417486_667080010060649?fields=reactions.summary(true),comments.limit(500){comment_count,like_count,created_time,from,message}&access_token=1577374979150890|OdmoPF-2_ZtrYw9eNICrBiGqr_4
PostID[313] 136845026417486_667080010060649 (reaction_by_id:2) time:10
314 136845026417486_666409383461045
https://graph.facebook.com/v2.6/136845026417486_666409383461045?fields=reactions.summary(true),comments.limit(500){comment_count,like_count,created_time,from,message}&access_token=1577374979150890|OdmoPF-2_ZtrYw9eNICrBiGqr_4
PostID[314] 136845026417486_666409383461045 (reaction_by_id:2) time:32
315 136845026417486_665903216844995
https://graph.facebook.com/v2.6/136845026417486_665903216844995?fields=reactions.summary(true),comments.limit(500){comment_count,like_count,created_time,from,message}&access_token=1577374979150890|OdmoPF-2_ZtrYw9eNICrBiGqr_4
PostID[315] 136845026417486_665903216844995 (reaction_by_id:2) time:98
316 

In [ ]:
#     Getting comments of comments
        maxcc = max(c['comment_count'] for c in comments)
        maxl = max(l['like_count']for l in likes)
        print maxcc
        if DEBUG: print('maxcc:%d'%maxcc)
        url = query_url(p['id'], 'fields=comments.limit(%d){comments.limit(%d)}'%(amount, maxcc), token)
        results = requests.get(url)
        data = json.loads(results.text)
        cannot_c2c = []
        if len(data.keys()) == 1:
            print "ERROR:CANNOT GET THE POST:", data
            cannot_c2c.append(p['id'])
            continue

        data = data['comments']
        c2cs = data['data']+likes_list
        if 'next' in data['paging']:
            while True:
                url = data['paging']['next']
                results = requests.get(url)
                data = json.loads(results.text)
                try:
                    c2cs += data['data']
                except:
                    time.sleep(3)
                    print "ERROR: time sleep 3 seconds"
                    print data.keys()
                    c2cs  += data['data']
                if 'next' not in data['paging']:
                    break
        print 'PostID[%d] %s (c2cs:%d) like_by_id:%d time:%d'%(i, p['id'], len(c2cs), len(likes_list),time.time()-start)
        print p['permalink_url']
        fname = directory + 'c2cment%d-%s.json'%(i, p['id'])
        with open(fname, 'w') as fout:
            json.dump(c2cs, fout)
